************ #Q1 ****************

In [1]:
from tqdm.auto import tqdm

C:\Users\ahmed\anaconda3\envs\encode\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [4]:
v= embedding_model.encode(user_question)

In [5]:
v[0]

0.07822266

In [6]:
len(v)

768

#Q1

In [7]:
v[:10]

array([ 0.07822266, -0.04013115,  0.03861359, -0.00017898,  0.08923472,
       -0.05045916, -0.01050265,  0.03710557, -0.04187143,  0.03480845],
      dtype=float32)

************ Q2 ****************

In [8]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [10]:
documents_f = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
len(documents_f)

375

In [11]:
documents_f[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [12]:
import numpy as np
embeddings = []
for doc in tqdm(documents_f):
    qa_text = f'{doc['question']} {doc['text']}'
    embd= embedding_model.encode(qa_text)
    embeddings.append(embd)
X = np.array(embeddings)

100%|████████████████████████████████████████████████████████████████████████████████| 375/375 [00:29<00:00, 12.50it/s]


In [13]:
X.shape

(375, 768)

********* Q3 **********

In [15]:
scores = X.dot(v)


In [16]:
max_values = round(max(scores),2)
max_values

0.65

***** Q4 Hit-rate for our search engine *******

In [34]:
v[:10]

array([ 0.07822266, -0.04013115,  0.03861359, -0.00017898,  0.08923472,
       -0.05045916, -0.01050265,  0.03710557, -0.04187143,  0.03480845],
      dtype=float32)

In [17]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents_f, embeddings=X)
#search_engine.search(v, num_results=5)

In [18]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [19]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [20]:
def vector_Engine_search(q):
    question = q['question']
    course = q['course']
    v_q = embedding_model.encode(question)

    return search_engine.search(v_q)

In [21]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return hit_rate(relevance_total)

In [22]:
#***** @4 results *****
evaluate(ground_truth, vector_Engine_search) #search_engine.search(v, num_results=5)

100%|██████████████████████████████████████████████████████████████████████████████| 1830/1830 [00:50<00:00, 36.17it/s]


0.9579234972677596

************ Q5 ****************

In [23]:
documents_f[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [28]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [25]:
from tqdm.auto import tqdm
#use filtered documents
for doc in tqdm(documents_f):
    question = doc['question']
    text = doc['text']
    qt = f'{question} {text}'

    doc['question_vector'] = embedding_model.encode(question)
    doc['text_vector'] = embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qt)

100%|████████████████████████████████████████████████████████████████████████████████| 375/375 [01:10<00:00,  5.34it/s]


In [29]:
for doc in tqdm(documents_f):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████| 375/375 [00:04<00:00, 83.66it/s]


In [59]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id", "score"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        score = hit['_score']
        result_docs.append(hit['_source'])

    return result_docs#, es_results

In [64]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']
    qt = f'{question} {text}'
    v_q = embedding_model.encode(question)

    #return elastic_search_knn('question_vector', v_q, course)
    return elastic_search_knn_combined(v_q, course)

In [61]:
########### Q5 answer
es_results = question_vector_knn(
    {'question': 'I just discovered the course. Can I still join?',
    'course':'machine-learning-zoomcamp'}
)
es_results[0]

{'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'section': 'General course-related questions',
 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'id': 'ee58a693'}

************ #Q6 ****************

In [67]:
def elastic_search_fn(q):
    question = q['question']
    course = q['course']
    results = question_vector_knn(q
    
    )

    return results

In [68]:
evaluate(ground_truth, elastic_search_fn) #search_engine.search(v, num_results=5)

100%|██████████████████████████████████████████████████████████████████████████████| 1830/1830 [02:45<00:00, 11.05it/s]


0.9218579234972678